In [10]:
import datasets
import abctk.obj.comparative as aoc

import ruamel.yaml

In [5]:
ds_raw = datasets.load_dataset(
    "abctreebank/comparative-NER-BCCWJ",
    revision = "2a9b95eaf4a78d4d195a06fcccbb77ac7526e2c4",
    use_auth_token = True,
)
ds: Dataset | IterableDataset = datasets.concatenate_datasets(
    [ds_raw["train"], ds_raw["test"]]
)

Using custom data configuration default-3d69770012c5f921
Found cached dataset parquet (/home/owner/.cache/huggingface/datasets/abctreebank___parquet/default-3d69770012c5f921/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
ds_new = []
for record in ds:
    record_new = dict(**record)
    record_new["annot"] = aoc.linearize_annotations(
        record["tokens"], record["comp"]
    )
    del record_new["tokens"]
    del record_new["comp"]
    ds_new.append(record_new)

In [20]:
yaml = ruamel.yaml.YAML()

def represent_annot(dumper, instance):
    if "[" in instance:
        return dumper.represent_scalar(
            "tag:yaml.org,2002:str",
            instance,
            style = "|"
        )
    else:
        return dumper.represent_scalar(
            "tag:yaml.org,2002:str",
            instance,
        )

yaml.representer.add_representer(str, represent_annot)
yaml.version = (1, 2)

In [21]:
with open("comparative-annotation_linearized_current.yaml", "w") as f:
    yaml.dump(ds_new, f)